In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

%matplotlib inline
pd.set_option('display.max_rows', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from modules.nypd_data import read_orig_file
from modules.nypd_data import filter_raw_data
from modules.nypd_data import save_dated_felonies
from modules.nypd_data import load_dated_felonies
from modules.nypd_data import save_clean_felonies
from modules.nypd_data import load_clean_felonies
from modules.nypd_data import add_offense_category
from modules.nypd_data import add_datetime_columns
from modules.nypd_data import pivot_felonies
from modules.nypd_data import save_pivoted_felonies
from modules.nypd_data import load_pivoted_felonies

In [3]:
orig_file = read_orig_file(None, 'NYPD_Complaint_Data_Current_YTD.csv')

In [4]:
filter_raw_data(orig_file, '../precrime_data/raw_recent_felonies.csv')

In [5]:
dated_felonies = load_dated_felonies(None, '../precrime_data/raw_recent_felonies.csv')

In [17]:
dated_felonies.to_csv('../precrime_data/clean_recent_felonies.csv', header=False)

In [8]:
clean_recent = load_clean_felonies(None, '../precrime_data/clean_recent_felonies.csv')
clean_old = load_clean_felonies()

/Users/jsamet/anaconda2/envs/py3k/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
clean_old.head()

,COMPLAINT_DATETIME,REPORT_DATE,KY_CD,OFNS_DESC,BORO_NM,ADDR_PCT_CD,Latitude,Longitude
CMPLNT_NUM,,,,,,,,
318229393,2006-01-02 00:00:00,2006-01-02,109,GRAND LARCENY,MANHATTAN,18,40.759529,-73.984397
554420424,2006-01-02 00:00:00,2008-11-26,112,THEFT-FRAUD,BROOKLYN,68,40.630222,-74.023848
218039893,2006-01-02 00:00:00,2006-11-26,109,GRAND LARCENY,MANHATTAN,6,40.733916,-74.000781
542037522,2006-01-02 00:00:00,2006-01-02,126,MISCELLANEOUS PENAL LAW,BROOKLYN,76,40.675672,-74.006256
492019346,2006-01-02 00:01:00,2006-01-07,109,GRAND LARCENY,BROOKLYN,70,40.634735,-73.952237


In [24]:
pd.merge(clean_old, clean_recent, left_index=True, right_index=True) # Making sure no duplicates

,COMPLAINT_DATETIME_x,REPORT_DATE_x,KY_CD_x,OFNS_DESC_x,BORO_NM_x,ADDR_PCT_CD_x,Latitude_x,Longitude_x,COMPLAINT_DATETIME_y,REPORT_DATE_y,KY_CD_y,OFNS_DESC_y,BORO_NM_y,ADDR_PCT_CD_y,Latitude_y,Longitude_y
CMPLNT_NUM,,,,,,,,,,,,,,,,


In [4]:
# At this point: I just appended the contents of clean_recent_felonies to the old clean_felonies file

nypd_data = load_clean_felonies()
add_offense_category(nypd_data)

/Users/jsamet/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [19]:
nypd_data.pivot_table(
    index=[
        nypd_data['COMPLAINT_DATETIME'].map(lambda x: x.year),
        nypd_data['COMPLAINT_DATETIME'].map(lambda x: x.month),
#        nypd_data['COMPLAINT_DATETIME'].map(lambda x: x.day),
#        nypd_data['COMPLAINT_DATETIME'].map(lambda x: x.hour),
#        'BORO_NM',
        'ADDR_PCT_CD',   # These are not duplicated across boros.
    ],
    values='KY_CD',
    columns='OFFENSE',
    fill_value=0,
    aggfunc=len
)

OFFENSE                                            Homicide  Rape  Robbery  \
COMPLAINT_DATETIME COMPLAINT_DATETIME ADDR_PCT_CD                            
2006               1                  1                   0     1        5   
                                      5                   0     2       16   
                                      6                   0     0       16   
                                      7                   1     1       17   
                                      9                   0     0       17   
                                      10                  0     2        7   
                                      13                  0     2       31   
                                      14                  0     2       29   
                                      17                  1     2        7   
                                      18                  0     0       13   
                                      19                  0     1       24   
                                      20                  0     2       26   
                                      22                  0     0        1   
                                      23                  0     2       26   
                                      24                  0     2       25   
                                      25                  1     2       28   
                                      26                  0     2       11   
                                      28                  3     1       27   
                                      30                  1     4       20   
                                      32                  1     4       34   
                                      33                  1     3       21   
                                      34                  0     2       28   
                                      40                  2     4       29   
                                      41                  0     3       25   
                                      42                  0     3       35   
                                      43                  0     5       37   
                                      44                  0     3       28   
                                      45                  1     1       20   
                                      46                  0     5       51   
                                      47                  2     7       41   
                                      48                  0     5       41   
                                      49                  1     4       24   
                                      50                  1     1       12   
                                      52                  3     3       44   
                                      60                  1     1       32   
                                      61                  1     3       26   
                                      62                  0     1       13   
                                      63                  2     2       25   
                                      66                  0     2       13   
                                      67                  1     8       55   
                                      68                  0     1       15   
                                      69                  1     1       31   
                                      70                  1     5       71   
                                      71                  0     3       25   
                                      72                  1     2       15   
                                      73                  4     7       61   
                                      75                  0     5       74   
                                      76                  0     1        9   
                                      77                  0     2       34   
                      

In [7]:
nypd_data[nypd_data['OFFENSE'] == 'Rape']

,COMPLAINT_DATETIME,REPORT_DATE,KY_CD,OFNS_DESC,BORO_NM,ADDR_PCT_CD,Latitude,Longitude,OFFENSE
CMPLNT_NUM,,,,,,,,,
679636523,2006-01-02 07:00:00,2006-02-02,104,RAPE,BROOKLYN,73,NaN,NaN,Rape
999299719,2006-01-02 09:00:00,2006-07-27,116,SEX CRIMES,BRONX,52,NaN,NaN,Rape
183691926,2006-01-02 09:00:00,2006-01-04,116,SEX CRIMES,BROOKLYN,79,NaN,NaN,Rape
804000472,2006-01-02 15:00:00,2006-01-02,104,RAPE,BROOKLYN,83,NaN,NaN,Rape
141882369,2006-01-02 17:00:00,2006-01-12,104,RAPE,STATEN ISLAND,122,NaN,NaN,Rape
698798201,2006-01-02 20:40:00,2006-01-02,116,SEX CRIMES,QUEENS,104,NaN,NaN,Rape
237796058,2006-01-02 22:30:00,2006-01-03,116,SEX CRIMES,MANHATTAN,26,NaN,NaN,Rape
789270555,2006-01-03 00:01:00,2006-02-13,104,RAPE,BRONX,45,NaN,NaN,Rape
189609347,2006-01-03 00:01:00,2006-03-17,104,RAPE,QUEENS,114,NaN,NaN,Rape


In [20]:
add_datetime_columns(nypd_data)

In [21]:
save_pivoted_felonies(nypd_data)

In [22]:
# Before running: execute "tar -xvf pivoted_felonies.tar.gz" from the root directory of the git repo
#
# To make the files from scratch (not necessary):
# save_pivoted_felonies(nypd_data)
nypd_pivoted = load_pivoted_felonies()

/Users/jsamet/anaconda2/envs/py3k/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [23]:
nypd_pivoted

COMPLAINT_DAYOFWEEK  \
COMPLAINT_YEAR COMPLAINT_MONTH COMPLAINT_DAY COMPLAINT_HOURGROUP ADDR_PCT_CD                        
2006           1               2             0                   1                              0   
                                                                 5                              0   
                                                                 6                              0   
                                                                 7                              0   
                                                                 9                              0   
                                                                 10                             0   
                                                                 13                             0   
                                                                 14                             0   
                                                                 17                             0   
                                                                 18                             0   
                                                                 19                             0   
                                                                 20                             0   
                                                                 22                             0   
                                                                 23                             0   
                                                                 24                             0   
                                                                 25                             0   
                                                                 26                             0   
                                                                 28                             0   
                                                                 30                             0   
                                                                 32                             0   
                                                                 33                             0   
                                                                 34                             0   
                                                                 40                             0   
                                                                 41                             0   
                                                                 42                             0   
                                                                 43                             0   
                                                                 44                             0   
                                                                 45                             0   
                                                                 46                             0   
                                                                 47                             0   
                                                                 48                             0   
                                                                 49                             0   
                                                                 50                             0   
                                                                 52                             0   
                                                                 60                             0   
                                                                 61                             0   
                                                                 62                             0   
                                                                 63                             0   
                                      